In [2]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None

In [3]:
# original data comes from https://www.kaggle.com/drgilermo/nba-players-stats
player_data = pd.read_csv('data/player_data.csv')
season_stats = pd.read_csv('data/Seasons_Stats.csv')
season_stats = season_stats.rename(columns = {'Unnamed: 0': 'origindex'}).drop(columns=['blanl', 'blank2'])
season_stats = season_stats.dropna()

In [4]:
season_stats = season_stats.dropna(subset=['Player'])
season_stats['Name'] = season_stats['Player'].replace(regex='\*$', value='')
season_stats['hof'] = season_stats['Player'].str.contains("\*")
season_stats = season_stats.drop(columns=['Player'])



In [5]:
total_yrs = season_stats.loc[season_stats['Tm'] == 'TOT', ['Year', 'Name']]
a = total_yrs.merge(season_stats, how='inner', on=['Year', 'Name'])
b = season_stats.drop(season_stats[season_stats['origindex'].isin(a[a['Tm'] != 'TOT']['origindex']) == True].index)
# drop players who don't have more than 5 seasons
c = b.groupby('Name').filter(lambda x: len(x) > 5)
d = pd.get_dummies(c, columns=['Pos', 'Tm'])

In [6]:
samples = len(d['Name'].unique())
timesteps = 20
features = 107

    
flat_data = np.zeros((samples, 2 + (timesteps * features)))
names = np.empty((samples),np.object)
for i, (name, group) in enumerate(d.groupby('Name')):
    df = group.copy()
    df = df.reset_index()
    #df['timestep'] = df.index
    oIndex = df.loc[0,['origindex']]
    
    label = df.loc[0,'hof']
    names[i] = name
    df = df.drop(columns=['index', 'origindex', 'Name', 'hof'])
    npflat = df.head(20).to_numpy().flatten()
    ids = np.array([i, label])
    row = np.concatenate([ids, npflat])
    flat_data[i][:len(row)]= row



In [11]:
hofindex = np.where(flat_data[:,1] == 1)[0]
hofrows = flat_data[hofindex]
hofnames = names[hofindex]
extrahof = flat_data
extrahofnames = names
for i in range(1, 11):
    extrahof = np.concatenate([extrahof, hofrows])
    extrahofnames = np.concatenate([extrahofnames, hofnames])

895 + 35 * 10 = 1245
895 + 35 * 10 = 1245


In [12]:
np.save('./data/flat_data.npy', flat_data)
np.save('./data/flat_names.npy', names)
np.save('./data/flat_data_extra.npy', extrahof)
np.save('./data/flat_names_extra.npy', extrahofnames)
print('done')